# Objective

The objective is to analyze the daily count of vaccinations in the top 5 countries in terms of the total vaccinations.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import style

style.use('dark_background')

font = {'family' : 'sans-serif',
        'weight' : 'bold',
        'size'   : 20}

plt.rc('font', **font)
plt.rc('xtick', labelsize=20) 
plt.rc('ytick', labelsize=20) 

In [ ]:
df = pd.read_csv('/kaggle/input/covid-world-vaccination-progress/country_vaccinations.csv', parse_dates = [2])

In [ ]:
df.head()

In [ ]:
df.shape

The following cell demonstrates the top 5 countries (in decreasing order) in terms of the total number of vaccinations till date.

In [ ]:
df.groupby('country')['total_vaccinations'].agg(max).nlargest(5)

In [ ]:
some_values = ['United States', 'China','United Kingdom', 'England', 'India']

vaccines = df.loc[df['country'].isin(some_values)]

In [ ]:
vaccines.fillna(0.0, inplace = True)

In [ ]:
vaccines.isna().sum()

Declaring functions to group the data by country and day.

In [ ]:
def GroupByCountryAndDay(df):
    groups = df.groupby('country')
    dailies = {}
    for name, group in groups:
        dailies[name] = GroupByDay(group)
        
    return dailies

def GroupByDay(df, func = np.mean):
    grouped = df[['date', 'daily_vaccinations']].groupby('date')
    daily = grouped.aggregate(func)
    daily['date'] = daily.index
    start = daily.date[0]
    one_year = np.timedelta64(1, 'Y')
    daily['years'] = (daily.date - start) / one_year
    return daily

In [ ]:
dailies = GroupByCountryAndDay(vaccines)

In [ ]:
dailies.keys()

The keys of the dictionary (dailies) is not in the order that we want that to be. Therefore, we create another dictionary in that specific order (the order demonstrated in 'some_values'). 

In [ ]:
reordered_dailies = {country: dailies[country] for country in some_values}

In [ ]:
reordered_dailies

# Visualization

In [ ]:
index = []
names = []
vacc = []

for i, (name, daily) in enumerate(reordered_dailies.items()):
    index.append(daily.index)
    names.append(name)
    vacc.append(daily.daily_vaccinations / 1000)

In [ ]:
names

In [ ]:
fig, axs = plt.subplots(5, figsize=(15,20))
fig.suptitle('Daily Vaccinations', fontsize = (20))
fig.autofmt_xdate(rotation = 30)

for i, (name, daily) in enumerate(reordered_dailies.items()):
    axs[i].scatter(index[i], vacc[i], color = 'red')
    axs[i].set_title(name)
    plt.tight_layout()